In [1]:
import io
from tqdm import tqdm
import numpy as np

def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in tqdm(fin):
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = np.array(list(map(float, tokens[1:])))
    return data


word2vec = load_vectors('wiki-news-300d-1M-subword.vec')


999994it [01:08, 14530.42it/s]


In [2]:

for item in word2vec:
    print(item)
    print(word2vec[item])
    print(word2vec[item].shape)
    print(word2vec[item] + word2vec[item])
    print((word2vec[item] + word2vec[item]) / 2)
    break

,
[ 2.0300e-02 -1.2300e-02 -7.6000e-03  1.8700e-02  1.7300e-02 -5.2000e-03
  1.4700e-02  1.3640e-01 -4.3500e-02  3.3000e-03 -2.5500e-02 -3.1100e-02
  3.0700e-02 -8.8000e-03 -1.0800e-02 -3.1000e-03 -1.4000e-03  1.3800e-02
 -2.0600e-02  7.4000e-03  2.1000e-02 -1.3800e-02  8.5900e-02  4.2500e-02
  3.7000e-03  3.9400e-02  2.7700e-02  1.7400e-02 -2.3200e-02  8.3000e-03
 -1.4200e-02 -2.7800e-02  4.1000e-03  8.0700e-02  1.1490e-01  7.8000e-02
 -9.4400e-02  5.2000e-03 -4.8000e-03 -1.6700e-02 -3.6200e-02 -6.0100e-02
  8.2000e-03  2.6400e-02 -5.3000e-03  2.6000e-02 -2.7000e-03 -4.0300e-02
  4.2000e-02  3.0400e-02 -5.4200e-02 -2.0200e-02  3.4900e-02  1.0000e-02
  9.1700e-02  4.8700e-02  6.6900e-02 -1.8200e-02  3.5000e-03 -4.1800e-02
  1.6000e-02 -2.3600e-02 -3.1840e-01  2.3400e-02  7.6600e-02 -2.0000e-04
  1.5800e-02 -1.4500e-02 -3.2800e-02 -2.4100e-02 -5.7900e-02  2.0000e-02
  2.3900e-02 -1.7400e-02 -1.4700e-02 -2.3500e-02  5.2500e-02  2.4410e-01
  5.1000e-03 -1.0000e-04 -1.6100e-02 -1.4700e-01 

In [3]:
sen2vec = {}
id2des = {}
desc_file_name = "data/SemEval-Food/semeval_food.desc"
node_file_name = "data/SemEval-Food/semeval_food.terms"
with open(desc_file_name, "r", encoding='utf-8') as fdesc:
    with open(node_file_name, "r", encoding='utf-8') as fin:
        for line, desc in tqdm(zip(fin, fdesc), desc="Loading terms"):
            line = line.strip()
            desc = desc.strip()
            if line:
                segs = line.split("\t")
                segs_desc = desc.split("\t")
                assert len(segs) == 2, f"Wrong number of segmentations {line}"
                try:
                    assert segs[1] == segs_desc[0]
                    desc = segs_desc[1]
                except AssertionError:
                    # assert len(segs_desc) == 1
                    desc = segs_desc[0]
                sen2vec[segs[0]] = segs[1]
                id2des[segs[0]] = desc
            #     print(sen2vec)
            #     print(id2des)
            # break

Loading terms: 1486it [00:00, 362326.23it/s]


In [4]:
def get_sen_emb(sentence):
    names = sentence.strip().split()
    tmp_vec = []
    for name in names:
        if name not in word2vec:
            continue
        tmp_vec.append(word2vec[name])
    if len(tmp_vec) == 0:
        tmp_vec = np.zeros(300)
        return tmp_vec
    tmp_vec = np.stack(tmp_vec)
    tmp_vec = np.mean(tmp_vec,0)
    return tmp_vec

In [ ]:
with open('data/SemEval-Food/semeval_food.terms.embed','w') as f:
    # print(len(sen2vec))
    now_line = f'{len(list(sen2vec.keys()))} 300\n'
    f.write(now_line)

    # x = 0
    # y = 0 
    lines = []
    a = 0
    for key in id2des:
        name = id2des[key]
        raw_name = name
        vec = get_sen_emb(raw_name)

        now_line = ''
        now_line += str(key)
        now_line += '\t'

        line = key + ' ' + ' '.join(str(i) for i in vec)+'\n'
        lines.append(line)
    f.writelines(lines)
    print(a)
        # norm_name = ''
        # for n in name:
        #     norm_name += n
        # print(norm_name)
        # if raw_name in word2vec and norm_name not in word2vec:
        #     x += 1
        # if norm_name not in word2vec and norm_name in word2vec:
        #     y += 1

In [5]:
with open('data/SemEval-Food/semeval_food.terms.name.embed','w') as f:
    # print(len(sen2vec))
    now_line = f'{len(list(sen2vec.keys()))} 300\n'
    f.write(now_line)

    # x = 0
    # y = 0 
    lines = []
    a = 0
    for key in sen2vec:
        name = sen2vec[key]
        raw_name = name
        vec = get_sen_emb(raw_name)

        now_line = ''
        now_line += str(key)
        now_line += '\t'

        line = key + ' ' + ' '.join(str(i) for i in vec)+'\n'
        lines.append(line)
    f.writelines(lines)
    print(a)
        # norm_name = ''
        # for n in name:
        #     norm_name += n
        # print(norm_name)
        # if raw_name in word2vec and norm_name not in word2vec:
        #     x += 1
        # if norm_name not in word2vec and norm_name in word2vec:
        #     y += 1

0


In [ ]:
print(id)